In [2]:
import pandas as pd
import numpy as np
import ast
import math
from pyvis.network import Network
import sys
sys.path.append('../')
from modules import utils, constants

#### The test df

In [3]:
model_name = 'dueling_ddqn_per'
seed = 105
steps = int(10e7)

In [4]:
test_df = pd.read_csv(f'../test_dfs/{model_name}_biopsy_3_best_path_pahm_{seed}_{steps}.csv')
test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,6.0,0.833333,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
1,1.0,11.0,0.666667,0.0,0.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
2,2.0,6.0,0.833333,1.0,1.0,"['ana', 'pericardial_effusion', 'joint_involve...",0.0,1.0
3,3.0,11.0,0.666667,0.0,0.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
4,4.0,12.0,0.633333,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0


In [5]:
utils.test(test_df.y_actual, test_df.y_pred)[0]

95.61428571428571

In [6]:
test_df[(test_df.y_pred==0) & (test_df.episode_length!=2)].episode_length.mean()

16.826662742789875

#### The other pertinent dfs

In [7]:
testing_df = pd.read_csv('../new_data/test_set_constant.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,4,1,0,0,0,1,0,0,1


In [8]:
training_df = pd.read_csv('../new_data/train_set_basic.csv')
training_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
1,1,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,1,0,...,1,3,0,0,0,0,0,0,0,1


#### Possibly useful functions

In [9]:
def generate_tuple_dict(df):
    frequency_dict = {}
    for traj in df.trajectory:
        if traj in frequency_dict.keys():
            frequency_dict[traj] += 1
        else:
            frequency_dict[traj] = 1
    overall_tup_dict = {}
    for key, value in frequency_dict.items():
        new_key = ast.literal_eval(key)
        for tup in zip(new_key, new_key[1:]):
            if tup in overall_tup_dict.keys():
                overall_tup_dict[tup] += value
            else:
                overall_tup_dict[tup] = value
    return overall_tup_dict

In [10]:
def create_sankey_df(df):
    overall_tup_dict = generate_tuple_dict(df)
    sankey_df = pd.DataFrame()
    sankey_df['source'] = [i[0] for i in overall_tup_dict.keys()]
    sankey_df['target'] = [i[1] for i in overall_tup_dict.keys()]
    sankey_df['value'] = list(overall_tup_dict.values())
    sankey_df['link_type'] = sankey_df['target'].apply(lambda i: 'terminal' if i in ['No lupus', 'Lupus', 'Inconclusive diagnosis'] else 'non_terminal')
    return sankey_df

In [11]:
def draw_pyvis_network(pathways_df, pathway_type='network'): #second type is trajectory 
    pathways_df['type'] = 'directed'
    start_node = pathways_df.iloc[0]['source']
    all_nodes = list(set(pathways_df.source.unique().tolist() + pathways_df.target.unique().tolist()))
    terminal_nodes = list(set(pathways_df[pathways_df.link_type=='terminal'].target))
    non_terminal_nodes = [i for i in all_nodes if i not in terminal_nodes and i!=start_node]
    
    #change get_net to something else since it satnds for Game of Thrones
    got_net = Network(notebook=True, height='600px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value, edge_thresh in zip(pathways_df.source, pathways_df.target, pathways_df.value, pathways_df.edge_threshold):
        if pathway_type =='network':
            if value > threshold:
                if math.isnan(edge_thresh):
                    got_net.add_edge(src, target, value=value, color='red')
                else:
                    got_net.add_edge(src, target, value=value, color='red', label=str(int(edge_thresh)))
            else:
                got_net.add_edge(src, target, value=value, color='blue')
        elif pathway_type =='trajectory': #It's just one trajectory
            if isinstance(edge_thresh, str):
                got_net.add_edge(src, target, value=value, color='blue', label=edge_thresh)
            else:
                got_net.add_edge(src, target, value=value, color='blue', label=str(int(edge_thresh)))
        else:
            print('Unknown pathway type')
    return got_net

In [12]:
def get_threshold_value(df, source): #source e.g. 'ana'
    threshold_values = df[source].unique().tolist()
    if len(threshold_values) == 1:
        return int(threshold_values[0])
    else:
        return str(threshold_values)

In [13]:
def draw_one_trajectory(pred_df, trajectory): #pred_df = pred_no_lupus par exemple
    trajectory_df = pred_df[pred_df.trajectory == trajectory]
    trajectory_testing_df = testing_df.loc[trajectory_df.index]
    trajectory_pathways_df = create_sankey_df(trajectory_df)
    trajectory_pathways_df['edge_threshold'] = [get_threshold_value(trajectory_testing_df, source) for source in trajectory_pathways_df.source]
    trajectory_net = draw_pyvis_network(trajectory_pathways_df, 'trajectory')   
    return trajectory_net

##### shortest pathway

In [14]:
pred_lupus = test_df[test_df['y_pred']==1]
shortest_lupus_path = pred_lupus[pred_lupus.episode_length == pred_lupus.episode_length.min()].trajectory.unique()[0]
shortest_lupus_path

"['ana', 'pericardial_effusion', 'joint_involvement', 'Lupus']"

In [15]:
len(pred_lupus[pred_lupus.trajectory ==shortest_lupus_path])

864

In [16]:
utils.get_pathway_score(shortest_lupus_path)

37.5

In [17]:
shortest_lupus_path_net = draw_one_trajectory(pred_lupus, shortest_lupus_path)
shortest_lupus_path_net.show('Example.html')

Example.html


##### commonest pathway 

In [18]:
commonest_lupus_path = pred_lupus.trajectory.value_counts().index[0]
commonest_lupus_path

"['ana', 'pericardial_effusion', 'joint_involvement', 'Lupus']"

In [19]:
len(pred_lupus[pred_lupus.trajectory ==commonest_lupus_path])

864

In [20]:
utils.get_pathway_score(commonest_lupus_path)

37.5

In [21]:
commonest_lupus_path_net = draw_one_trajectory(pred_lupus, commonest_lupus_path)
commonest_lupus_path_net.show('Example.html')

Example.html


##### longest pathways 

In [22]:
len(pred_lupus[pred_lupus.episode_length == pred_lupus.episode_length.max()].trajectory.value_counts())

6

In [23]:
longest_lupus_path = pred_lupus[pred_lupus.episode_length==pred_lupus.episode_length.max()].trajectory.value_counts().index[0]
longest_lupus_path

"['ana', 'pericardial_effusion', 'anti_dsdna_antibody', 'proteinuria', 'acute_pericarditis', 'delirium', 'non_scarring_alopecia', 'pleural_effusion', 'leukopenia', 'joint_involvement', 'anti_cardioliphin_antibodies', 'low_c3', 'thrombocytopenia', 'seizure', 'oral_ulcers', 'lupus_anti_coagulant', 'cutaneous_lupus', 'auto_immune_hemolysis', 'fever', 'low_c4', 'anti_β2gp1_antibodies', 'anti_smith_antibody', 'psychosis', 'biopsy_proven_lupus_nephritis', 'Lupus']"

In [24]:
len(pred_lupus[pred_lupus.trajectory ==longest_lupus_path])

8

In [25]:
utils.get_pathway_score(longest_lupus_path)

283.5

#### delete from here

In [29]:
path_test_df = pred_lupus[pred_lupus.trajectory ==longest_lupus_path]
path_df = testing_df.loc[path_test_df.index]
path_df.biopsy_proven_lupus_nephritis.value_counts()

0    7
3    1
Name: biopsy_proven_lupus_nephritis, dtype: int64

In [30]:
path_test_df

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
348,348.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
3735,3735.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
5301,5301.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
5565,5565.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
6359,6359.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
11899,11899.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
12970,12970.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0
13012,13012.0,25.0,0.2,1.0,1.0,"['ana', 'pericardial_effusion', 'anti_dsdna_an...",0.0,1.0


In [40]:
path_df[((path_df.low_c4==1)&(path_df.psychosis==0))&(path_df.biopsy_proven_lupus_nephritis==3)]

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
13012,1,0,0,0,0,0,0,0,0,0,...,1,3,0,0,0,1,1,0,0,1


#### end here

In [26]:
longest_lupus_path_net = draw_one_trajectory(pred_lupus, longest_lupus_path)
longest_lupus_path_net.show('Example.html')

Example.html


In [ ]:
# len(pred_lupus[pred_lupus.trajectory ==longest_lupus_path2])

In [ ]:
# longest_lupus_path2_net = draw_one_trajectory(pred_lupus, longest_lupus_path2)
# longest_lupus_path2_net.show('Example.html')

In [ ]:
#### Misdi

In [ ]:
pathway_samp = str(['ana', 'joint_involvement', 'anti_dsdna_antibody', 'low_c3', 'psychosis', 'anti_smith_antibody', 'pleural_effusion', 'proteinuria', 'leukopenia', 'fever', 'non_scarring_alopecia', 'low_c4', 'pericardial_effusion', 'cutaneous_lupus', 'seizure', 'delirium', 'lupus_anti_coagulant', 'thrombocytopenia', 'Lupus'])
pathway_samp

In [ ]:
a = pred_lupus[(pred_lupus.trajectory == pathway_samp)&(pred_lupus.y_actual!=pred_lupus.y_pred)]

In [ ]:
draw_one_trajectory(pred_lupus, pathway_samp).show('Example.html')

In [ ]:
action = 5
reward = 0
print(constants.ACTION_SPACE[action])
print(constants.FEATURE_SCORES[constants.ACTION_SPACE[action]])
print(1.5*constants.FEATURE_SCORES[constants.ACTION_SPACE[action]])
reward += -1/(1.5*constants.FEATURE_SCORES[constants.ACTION_SPACE[action]])
reward

In [ ]:
1/18